# クロージャが変数スコープとどう関わるかを把握しておく

In [15]:
# この関数は、リスト values の内、group に含まれる値を
# 優先的に先頭へ並べるようにソートする関数です。
def sort_priority(values, group):
  def helper(x):
    if x in group:
      return (0, x)
    return (1, x)
  values.sort(key=helper)

In [16]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
sort_priority(numbers, group)
print(numbers)

[2, 3, 5, 7, 1, 4, 6, 8]


Python の sort は、タプル (a, b) の場合、まず a を見て、小さい順に並べます。
a が同じなら、次に b を見て並べます。
なので、ソートキー (0, x) は優先され、(1, x) は後回し。

In [17]:
tuples = [(0,3), (0,2), (0,1), (1,11), (1, 13), (1, 12)]
tuples.sort()
print(tuples)

[(0, 1), (0, 2), (0, 3), (1, 11), (1, 12), (1, 13)]


In [ ]:
def sort_priority(values, group):
  found = False
  def helper(x):
    if x in group:
      # print(found) # 動作しない 以下の例外がスローされる
      # UnboundLocalError: cannot access local variable 'found' where it is not associated with a value
      found = True # この found は ↑ で定義した found とは別の変数として扱っている
      # print(found) # 動作する True が表示
      return (0, x)
    # print(found) # 動作しない 以下の例外がスローされる
    # UnboundLocalError: cannot access local variable 'found' where it is not associated with a value
    return (1, x)
  # print(found) # 動作する False が表示
  values.sort(key=helper)
  # リスト values の内、group に含まれる値があった場合、
  # found = True が返り値になることを期待
  return found

In [43]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority(numbers, group)
# sort は期待通り動いている
print(numbers)
# が、found = False が返ってくる
print(found)

[2, 3, 5, 7, 1, 4, 6, 8]
False


In [53]:
# データをクロージャの外に出す特別な構文 nonlocal がある。
# 指定した変数名の代入に際してスコープを横断する
# 唯一の制限は、nonlocal が（グローバルを汚染しないように）モジュールレベルのスコープまでは行かない
def sort_priority(values, group):
  found = False
  def helper(x):
    nonlocal found # 追加
    if x in group:
      found = True
      return (0, x)
    return (1, x)
  values.sort(key=helper)
  return found

In [54]:
numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = {2, 3, 5, 7}
found = sort_priority(numbers, group)
print(numbers)
# 期待通り found = True が返ってくる
print(found)

[2, 3, 5, 7, 1, 4, 6, 8]
True


ただし、nonlocal を単純な関数以外で利用することは良くない！

見つけずらくなるから！

ので、ヘルパークラスでラップするとよい

In [55]:
class Sorter:
  def __init__(self, group):
    self.group = group
    self.found = False

  def __call__(self, x):
    if x in self.group:
      self.found = True
      return (0, x)
    return (1, x)

In [58]:
sorter = Sorter(group)
numbers.sort(key=sorter)
assert sorter.found is True